# Work Item Tracking with Python
This notebook provides examples of how to query Azure DevOps work items using the Azure DevOps python library.

## Prerequisites
Before executing this notebook, install the devops library using

```
pip install azure-devops
```

In [311]:
from azure.devops.connection import Connection
from msrest.authentication import BasicAuthentication
from azure.devops.v5_1.work_item_tracking.models import Wiql
from azure.devops.v5_1.work_item_tracking.models import TeamContext

## Connect to DevOps

You need a personal access token from DevOps to access work items from your python program

In [312]:
token = "<token>"
team_instance = "https://msdata.visualstudio.com"

credentials = BasicAuthentication("PAT", token)
connection = Connection(base_url=team_instance, creds=credentials)

The WorkItemTrackingClient is the main object for accessing all information about the work items.  We are using the preview 5.1 client.  There are significant differences in API's between versions, so changing the version will mean that you will most likely need to rework the code.

In [313]:
# the client we need is not the default, so we get a specific client
wit_client = connection.get_client('azure.devops.v5_1.work_item_tracking.work_item_tracking_client.WorkItemTrackingClient')

## Query Work Items
This query is a "flat query" in that is returns a standard non-hierarchical list.  Working with heirarchical lists is possible but requires experimentation to determine how to interpret the results.  If you want to create a hierarchical query, it is recommended to create the query using the DevOps user interface and copying the resulting WIQL.

Note that the query *projection*, ie the list of columns, is essentially ignored.  The query result includes the list of ids filtered and orderd by the query, along with some other information about the query, but the actual fields are retrieved in later calls.

In [315]:
flatquery = """
SELECT
        [System.Id],
        [System.WorkItemType],
        [System.Title],
        [System.State],
        [System.AreaPath],
        [System.IterationPath]
FROM workitems
WHERE
        [System.TeamProject] = @project
        AND [System.ChangedDate] > @today - 7
ORDER BY [System.ChangedDate] DESC
"""

Executing the query by converting to wiql and executing against the client.  The project can be set via the Team Context or directly in the query.  DevOps returns a maximum of 20,000 workitem references.

In [316]:
query_wiql = Wiql(query=flatquery)
query_result = results = wit_client.query_by_wiql(query_wiql, TeamContext(project="Vienna"))
work_item_references = query_result.work_items

## Process the results

Since the query only returns ids, we need to process the ids and fetch the actual work item data

First, extract the ids from the references that are returned so that we can retrieve work items in batch

In [322]:
ids=list(wir.id for wir in work_item_references)

Each work item is retrieved with a default set of fields including Title, AreaPath, IterationPath, etc.  This can be overridded by using the `fields` option in the `get_work_items` call, and providing a string array listing the fields of interest.  In this case, however, we are using the `expand` option to retrieve the related work items.  Using both of these options is incompatible, so if you want to retrieve non-default fields and related work items, you will need to pull the work items twice.

This code retrieves the workitems in batches.  The maximum batch size is 200.  

In [357]:
id_start = 0
batch_size = 200
work_items=[]
while id_start < len(ids) :
    work_items.extend(wit_client.get_work_items(ids[id_start:id_start+batch_size], expand="Relations"))
    id_start += batch_size
    print (id_start)
print ("Read {} work items from {} ids".format(len(work_items), len(ids)))

200
400
600
800
1000
1200
1400
1600
1800
2000
Read 1980 work items from 1980 ids


## Understanding the results

These are the fields that are returned by default 

In [382]:
wi = work_items[0]
for key in wi.fields.keys():
    print(key)

System.AreaPath
System.TeamProject
System.IterationPath
System.WorkItemType
System.State
System.Reason
System.AssignedTo
System.CreatedDate
System.CreatedBy
System.ChangedDate
System.ChangedBy
System.CommentCount
System.Title
System.BoardColumn
System.BoardColumnDone
Microsoft.VSTS.Common.StateChangeDate
Microsoft.VSTS.Common.ActivatedDate
Microsoft.VSTS.Common.ActivatedBy
Microsoft.VSTS.Common.Priority
Microsoft.VSTS.Common.ValueArea
WEF_1366634518964FB0850FD25BC19879F0_Kanban.Column
WEF_1366634518964FB0850FD25BC19879F0_Kanban.Column.Done
WEF_193A890692DB4476803F6DCBF5FF4B7C_Kanban.Column
WEF_193A890692DB4476803F6DCBF5FF4B7C_Kanban.Column.Done
WEF_5AA31F69748743F5B2FD63F7D3497E2A_Kanban.Column
WEF_5AA31F69748743F5B2FD63F7D3497E2A_Kanban.Column.Done
WEF_400060F5BD1849B0B5C727AB83F5B92C_Kanban.Column
WEF_400060F5BD1849B0B5C727AB83F5B92C_Kanban.Column.Done
System.Description


Assuming there are any related items, this cell will display the structure of the first relation (or throw if there are none)

In [377]:
try:
    print(wi.relations[0].as_dict())
except:
    print("no relations")

{'attributes': {'isLocked': False, 'name': 'Child'}, 'rel': 'System.LinkTypes.Hierarchy-Forward', 'url': 'https://msdata.visualstudio.com/3adb301f-9ede-41f2-933b-fcd1a486ff7f/_apis/wit/workItems/443983'}


This cell will display the ID of the parent work item, if any

In [380]:
try:
    parent = next((relation for relation in wi.relations if relation.rel == 'System.LinkTypes.Hierarchy-Reverse'))
    print( parent.attributes['name'], parent.url.rsplit('/', 1)[-1])
except StopIteration:
    print("no parent")

Parent 443972


## Fetching comments
Comments are not included in the work item records and need to be fetched individually.

This cell will fetch and display the most recent comment, if any.  Note that the output is an HTML rendering of the text in the latest comment.

In [ ]:
try:
    comment = wit_client.get_comments(project="Vienna", work_item_id = wi.id , top=1, order="desc").comments[0].text
except:
    comment = ""
comment
